In [12]:
!pip install dataclasses
!pip install typing
!pip install uuid
!pip install datetime
!pip install json

zsh:1: command not found: pip
zsh:1: command not found: pip
zsh:1: command not found: pip
zsh:1: command not found: pip
zsh:1: command not found: pip


In [13]:
# Constants
# Role Constants
ROLE_A = "A" 
ROLE_B = "B"
ROLE_C = "C"
ROLE_D = "D"
ROLE_E = "E"
ROLE_F = "F"
ROLE_G = "G"
ROLE_H = "H"
ROLE_I = "I"

# Functions and Positions
FUNC_FINANCIAL_ANALYST = "Financial Analyst"
FUNC_SHARE_TECHNICIAN = "Share Technician"
FUNC_SUPPORT_ECOMM = "Support Ecomm"
FUNC_OFFICE_BANKING = "Office Banking"

POS_CLERK = "Clerk"
POS_JUNIOR_ANALYST = "Junior Analyst"
POS_SENIOR_ANALYST = "Senior Analyst"
POS_SPECIALIST = "Specialist"
POS_GROUP_MANAGER = "Group Manager"
POS_HEAD_DIVISION = "Head of Division"
POS_JUNIOR = "Junior"  # For Support Ecomm

# Applications
APP_MMI       = "Money Market Instruments"
APP_DERIV     = "Derivatives"
APP_INTEREST  = "Interest Rates"
APP_CONSUMER  = "Consumer Banking"
APP_IB        = "Investment Banking"
APP_TREASURY  = "Treasury Management"
APP_LOANS     = "Loan Management"
APP_RISK      = "Risk Management"
APP_CORP      = "Corporate Banking"
APP_SHARE     = "Share Trading"
APP_ECOMM     = "E-Commerce"
APP_ADMIN     = "Administration"

# Permissions
PERM_READ    = "Read"
PERM_WRITE   = "Write"
PERM_EXECUTE = "Execute"
PERM_APPROVE = "Approve"
PERM_AUDIT   = "Audit"
PERM_REPORT  = "Report"
'''
User
  └── Position (e.g., Clerk)
        └── Function (e.g., Financial Analyst)
              └── Role (e.g., A, B, G)
                    └── Permissions (e.g., view, edit, approve) based on Applications
'''
from dataclasses import dataclass, field
from typing import List, Set, Dict
import uuid
from datetime import datetime, timedelta, timezone
import json

# Data classes for RBAC (Task A)
@dataclass
class Role:
    """
    Represents an RBAC Role, which may inherit from one or more other roles.
    """
    name: str
    inherits_from: List[str] = field(default_factory=list) # Task A-c: Role hierarchy

@dataclass
class Function:
    """
    Represents a departental function.
    Each function can support multiple roles.
    """
    name: str
    roles: List[str] # Role names

@dataclass
class Position:
    """
    Represents an official position like Clerk and Head of Division.
    Linked to a function and assigned a default or primary role.
    """
    title: str
    function_name: str # Link to a Function
    assigned_role: str # Task A-b: Map users to one or more roles

@dataclass
class Application:
    """
    Represents an application in the organization that can have different permissions.
    """
    name: str

@dataclass
class PermissionBinding:
    """
    Links a specific Role to an Application with certain access rights.
    For example, Role A for Money Market Instruments has {Read, Write}.
    """
    role: Role
    application: Application
    access_rights: Set[str]


def get_all_inherited_roles(role_name: str, role_dict: Dict[str, Role]) -> Set[str]:
    """
    Recursively gather all role names inherited by 'role_name'.
    For example, if Role C inherits B, and B inherits A, then C includes A, B, and C.

    Args:
        role_name (str): The name of the role whose inherited hierarchy should be resolved.
        role_dict (Dict[str, Role]): A dictionary mapping role names to Role objects,
                                     used to look up inheritance relationships.

    Returns:
        Set[str]: A set containing the given role and all roles it inherits from.
    """
    roles = {role_name}
    role = role_dict.get(role_name)
    if role:
        for parent in role.inherits_from:
            roles.update(get_all_inherited_roles(parent, role_dict))
    return roles

@dataclass
class User:
    """
    The base User class (without credentials).
    This class stores an internal user_id,
    their position title, and the function name they belong to.
    """
    user_id: str
    position_title: str   
    function_name: str       

# Role Dictionary based on table 3
role_dict = {
    ROLE_A: Role(name=ROLE_A, inherits_from=[]),
    ROLE_B: Role(name=ROLE_B, inherits_from=[ROLE_A]),
    ROLE_C: Role(name=ROLE_C, inherits_from=[ROLE_B]),
    ROLE_D: Role(name=ROLE_D, inherits_from=[ROLE_C]),
    ROLE_E: Role(name=ROLE_E, inherits_from=[ROLE_D]),
    ROLE_F: Role(name=ROLE_F, inherits_from=[ROLE_E]),
    ROLE_G: Role(name=ROLE_G, inherits_from=[]),
    ROLE_H: Role(name=ROLE_H, inherits_from=[]),
    ROLE_I: Role(name=ROLE_I, inherits_from=[])
}

# Application Dictionary based on the 2
applications_dict = {
    APP_MMI: Application(name=APP_MMI),
    APP_DERIV: Application(name=APP_DERIV),
    APP_INTEREST: Application(name=APP_INTEREST),
    APP_CONSUMER: Application(name=APP_CONSUMER),
    APP_IB: Application(name=APP_IB),
    APP_TREASURY: Application(name=APP_TREASURY),
    APP_LOANS: Application(name=APP_LOANS),
    APP_RISK: Application(name=APP_RISK),
    APP_CORP: Application(name=APP_CORP),
    APP_SHARE: Application(name=APP_SHARE),
    APP_ECOMM: Application(name=APP_ECOMM),
    APP_ADMIN: Application(name=APP_ADMIN)
}

# Permission Bindings (Role -> App -> Permissions); Table 2
bindings = [
    PermissionBinding(role_dict[ROLE_A], applications_dict[APP_MMI], {PERM_READ, PERM_WRITE}),
    PermissionBinding(role_dict[ROLE_A], applications_dict[APP_DERIV], {PERM_READ, PERM_EXECUTE}),
    PermissionBinding(role_dict[ROLE_A], applications_dict[APP_INTEREST], {PERM_READ, PERM_EXECUTE}),
    PermissionBinding(role_dict[ROLE_B], applications_dict[APP_MMI], {PERM_READ, PERM_WRITE, PERM_EXECUTE}),
    PermissionBinding(role_dict[ROLE_B], applications_dict[APP_DERIV], {PERM_READ, PERM_WRITE, PERM_EXECUTE}),
    PermissionBinding(role_dict[ROLE_B], applications_dict[APP_CONSUMER], {PERM_READ, PERM_WRITE}),
    PermissionBinding(role_dict[ROLE_C], applications_dict[APP_IB], {PERM_READ, PERM_WRITE, PERM_APPROVE}),
    PermissionBinding(role_dict[ROLE_C], applications_dict[APP_TREASURY], {PERM_READ, PERM_WRITE, PERM_EXECUTE}),
    PermissionBinding(role_dict[ROLE_D], applications_dict[APP_LOANS], {PERM_READ, PERM_WRITE, PERM_EXECUTE}),
    PermissionBinding(role_dict[ROLE_E], applications_dict[APP_RISK], {PERM_READ, PERM_WRITE, PERM_EXECUTE, PERM_APPROVE}),
    PermissionBinding(role_dict[ROLE_F], applications_dict[APP_CORP], {PERM_READ, PERM_WRITE, PERM_EXECUTE, PERM_AUDIT, PERM_REPORT}),
    PermissionBinding(role_dict[ROLE_G], applications_dict[APP_SHARE], {PERM_READ, PERM_WRITE}),
    PermissionBinding(role_dict[ROLE_H], applications_dict[APP_ECOMM], {PERM_READ, PERM_WRITE, PERM_EXECUTE}),
    PermissionBinding(role_dict[ROLE_I], applications_dict[APP_ADMIN], {PERM_READ, PERM_WRITE, PERM_APPROVE, PERM_AUDIT}),
]

# Position Lookup based on table 1
position_lookup = {
    (FUNC_FINANCIAL_ANALYST, POS_CLERK): Position(title=POS_CLERK, function_name=FUNC_FINANCIAL_ANALYST, assigned_role=ROLE_A),
    (FUNC_FINANCIAL_ANALYST, POS_JUNIOR_ANALYST): Position(title=POS_JUNIOR_ANALYST, function_name=FUNC_FINANCIAL_ANALYST, assigned_role=ROLE_B),
    (FUNC_FINANCIAL_ANALYST, POS_SENIOR_ANALYST): Position(title=POS_SENIOR_ANALYST, function_name=FUNC_FINANCIAL_ANALYST, assigned_role=ROLE_C),
    (FUNC_FINANCIAL_ANALYST, POS_SPECIALIST): Position(title=POS_SPECIALIST, function_name=FUNC_FINANCIAL_ANALYST, assigned_role=ROLE_D),
    (FUNC_FINANCIAL_ANALYST, POS_GROUP_MANAGER): Position(title=POS_GROUP_MANAGER, function_name=FUNC_FINANCIAL_ANALYST, assigned_role=ROLE_E),
    (FUNC_FINANCIAL_ANALYST, POS_HEAD_DIVISION): Position(title=POS_HEAD_DIVISION, function_name=FUNC_FINANCIAL_ANALYST, assigned_role=ROLE_F),
    (FUNC_SHARE_TECHNICIAN, POS_CLERK): Position(title=POS_CLERK, function_name=FUNC_SHARE_TECHNICIAN, assigned_role=ROLE_G),
    (FUNC_SUPPORT_ECOMM, POS_JUNIOR): Position(title=POS_JUNIOR, function_name=FUNC_SUPPORT_ECOMM, assigned_role=ROLE_H),
    (FUNC_OFFICE_BANKING, POS_HEAD_DIVISION): Position(title=POS_HEAD_DIVISION, function_name=FUNC_OFFICE_BANKING, assigned_role=ROLE_I)
}

@dataclass
class AuthenticatedUser(User):
    """
    Represents an authenticated user with credentials and additional role assignments.

    Extends the base `User` class by adding:
    - A unique `username`
    - A `password` (for assignment purposes, stored as plaintext, normally it should be the hash)
    - Up to 4 role assignments, in addition to the default role based on position

    The user’s effective permissions are based on:
    - Their explicitly assigned roles (if any)
    - Their default role (from their Position)
    - All inherited roles

    Args:
        username (str): Login identifier for the user.
        password (str): Password string (ideally hashed in real systems).
        assigned_roles (List[str]): Optional list of directly assigned roles (max 4 including default).
    Raises:
        ValueError: If the total number of effective roles (assigned + default) exceeds 4.
    """
    username: str
    password: str
    assigned_roles: List[str] = field(default_factory=list)  # Direct role assignments

    def __post_init__(self):
        """
        Enforce a maximum of 4 assigned roles for Security Enhancement.

        Raises:
            ValueError: If the total number of unique roles (assigned + default) exceeds 4.
        """
        default_role = position_lookup.get((self.function_name, self.position_title)).assigned_role
        total_roles = set(self.assigned_roles)
        total_roles.add(default_role)

        if len(total_roles) > 4:
            raise ValueError(
                f"User cannot hold more than 4 roles (assigned + default). "
                f"Has: {len(total_roles)} → {total_roles}"
            )
    
    # Get effective permissions using the user's assigned roles (plus inheritance)
    def get_user_permissions(self, role_dict: Dict[str, Role], bindings: List["PermissionBinding"]) -> Dict[str, Set[str]]:
        """
        Gather all permissions for the user, including:
        - Their explicitly assigned roles
        - Their default role from position
        - All inherited roles
        Returns a mapping: {application name: set of permissions}

        Args:
            role_dict (Dict[str, Role]): All roles and their hierarchies
            bindings (List[PermissionBinding]): Role to app permission bindings

        Returns:
            Dict[str, Set[str]]: Application-wise permissions
        """
        all_permissions: Dict[str, Set[str]] = {}
        effective_roles = set(self.assigned_roles)

        # Always include the default role from position
        default_role = position_lookup.get((self.function_name, self.position_title)).assigned_role
        effective_roles.add(default_role)

        # Resolve inheritance (e.g., B → A → inherited permissions)
        all_inherited_roles = set()
        for role_name in effective_roles:
            all_inherited_roles.update(get_all_inherited_roles(role_name, role_dict))

        # Match permissions for all effective roles
        for binding in bindings:
            if binding.role.name in all_inherited_roles:
                all_permissions.setdefault(binding.application.name, set()).update(binding.access_rights)

        return all_permissions

@dataclass
class Session:
    """
    Represents a user's authenticated session with an actice role.
    Sessions expire after a fixed period (1 hour in this example).
    """
    session_id: str
    user_id: str
    active_role: str
    start_time: datetime
    expires_at: datetime

session_store: Dict[str, Session] = {}

def start_session(user_id: str, selected_role: str) -> str:
    """
    Starts a new session for the given user with a selected active role.
    
    The selected role must be one of:
    - The user's explicitly assigned roles, or
    - The default role tied to their position (function + title)

    Returns:
        session_id (str): The UUID of the session if successful, else "".
    """

    # If the user has no assigned roles, they must use the default role from position
    user = user_store.get(user_id)
    if not user:
        print(f"ERROR: User {user_id} not found.")
        return ""

    # Get all valid roles for the user (assigned + default)
    valid_roles = set(user.assigned_roles)
    default_role = position_lookup.get((user.function_name, user.position_title)).assigned_role
    valid_roles.add(default_role)

    # Check if selected role is allowed
    if selected_role not in valid_roles:
        print(f"ERROR: {user.user_id} cannot use role {selected_role}. Allowed roles: {valid_roles}")
        return ""

    # Create session
    session_id = str(uuid.uuid4())
    now = datetime.now()
    session_store[session_id] = Session(
        session_id=session_id,
        user_id=user_id,
        active_role=selected_role,
        start_time=now,
        expires_at=now + timedelta(hours=1)
    )
    print(f"Session started: {user.user_id} with active role {selected_role}. (Session ID: {session_id})")
    return session_id

access_log = []

def log_access(user_id: str, username: str, app_name: str, action: str, result: str):
    """
    Writes a single access attempt to 'access_log'. 
    Capturing:
    - Timestamp of the attempt (in GMT)
    - User ID and username
    - Target application
    - Requested action (e.g., "Read", "Write", etc.)
    - Outcome (e.g., "Access granted", "Access denied")
    
    Args:
        user_id (str): The unique ID of the user attempting access.
        username (str): The human-readable username.
        app_name (str): The name of the application being accessed.
        action (str): The permission type the user attempted (e.g., Read, Write).
        result (str): Outcome of the access check (granted or reason for denial).
    """
    timestamp = datetime.now(timezone.utc).strftime("%a, %d %b %Y %H:%M:%S GMT")
    log_entry = {
        "timestamp": timestamp,
        "user": username,
        "user_id": user_id,
        "application": app_name,
        "access_type": action,
        "result": result
    }
    access_log.append(log_entry)

def authorize_access_by_username(username: str, password: str, app_name: str, action: str) -> bool:
    """
    Authroization check without sessions: validates username/password,
    checks if the user has the necessary permission for 'action' in 'app_name'. 
    
    Logs the result in 'access_log'. 

    More like a testing function before implementing Session.

    Args:
        username (str): The login name of the user.
        password (str): The provided password (compared in plaintext; not recommended for production).
        app_name (str): The name of the application the user is trying to access.
        action (str): The type of access being requested (e.g., "Read", "Write").

    Returns:
        bool: True if access is granted, False otherwise.
    """
    user_id = username_index.get(username)
    if not user_id:
        result = f"Denied: Unknown username '{username}'"
        print(f"[AUTH LOG] {result}")
        log_access("N/A", username, app_name, action, result)
        return False

    user = user_store.get(user_id)
    if not user:
        result = "Denied: User ID not found"
    elif user.password != password:
        result = "Denied: Invalid credentials"
    else:
        permissions = user.get_user_permissions(role_dict, bindings)
        allowed_actions = permissions.get(app_name, set())
        if action in allowed_actions:
            result = "Access granted"
        else:
            result = f"Denied: '{action}' not allowed on '{app_name}'"
    
    log_access(user_id, username, app_name, action, result)
    print(f"[AUTH LOG] {username} ({user_id}) -> {app_name} -> {action} -> {result}")
    return "granted" in result.lower()

def authorize_session_access(session_id: str, app_name: str, action: str) -> bool:
    """
    Authorization check with a session: ensures the session is valid and not expired,
    retrieves the 'active_role' from the session, and checks if that role has permission for 
    'action' on 'app_name'.

    Args:
        session_id (str): The UUID of the active session.
        app_name (str): The name of the application the user wants to access.
        action (str): The type of action being requested (e.g., "Read", "Write", "Approve").

    Returns:
        bool: True if access is granted, False otherwise.
    """
    session = session_store.get(session_id)
    if not session:
        print("Access Denied: No active session.")
        return False

    now = datetime.now()
    if now > session.expires_at:
        print(f"ERROR: Session expired for {session.user_id}.")
        del session_store[session_id]
        return False

    user = user_store.get(session.user_id)
    if not user:
        print("Access Denied: User not found.")
        return False

    # Validate the active role — include fallback to default role if none are assigned
    if user.assigned_roles:
        if session.active_role not in user.assigned_roles:
            print(f"ERROR: {user.user_id} cannot use role {session.active_role}.")
            return False
    else:
        default_role = position_lookup.get((user.function_name, user.position_title)).assigned_role
        if session.active_role != default_role:
            print(f"ERROR: {user.user_id} cannot use role {session.active_role}. (Only allowed role: {default_role})")
            return False

    effective_roles = get_all_inherited_roles(session.active_role, role_dict)
    allowed_actions = set()
    for binding in bindings:
        if binding.role.name in effective_roles and binding.application.name == app_name:
            allowed_actions.update(binding.access_rights)

    if action in allowed_actions:
        result = "Granted"
        print(f"Access Granted: {user.user_id} (Role {session.active_role}) -> {app_name} ({action})")
    else:
        result = f"Access Denied: {user.user_id} (Role {session.active_role}) -> {app_name} ({action})"
        print(result)

    log_access(user.user_id, user.username, app_name, action, result)
    return "granted" in result.lower()


# Example User Store
user_store: Dict[str, AuthenticatedUser] = {
    "Auth001": AuthenticatedUser(
        user_id="Auth001",
        username="alice",
        password="password",
        position_title=POS_CLERK,
        function_name=FUNC_FINANCIAL_ANALYST,
        assigned_roles=[ROLE_B]  # alice's roles (subset of allowed_roles for Clerk in Financial Analyst)
    ),
    "Auth002": AuthenticatedUser(
        user_id="Auth002",
        username="bob",
        password="password",
        position_title=POS_HEAD_DIVISION,
        function_name=FUNC_OFFICE_BANKING,
        assigned_roles=[]  # bob's roles (subset of allowed_roles for Head Division in Office Banking)
    ),
}

# Username to UserID index
username_index: Dict[str, str] = { 
    "alice": "Auth001",
    "bob": "Auth002"
}


In [14]:
# Test cases
print("=== Creating Users ===")
for uid, user in user_store.items():
    print(f"Created User: {user.user_id}, Username: {user.username}, Position: {user.position_title}, Function: {user.function_name}, Assigned Roles: {user.assigned_roles}")
print("\n=== Testing Access Control by Username (No Session) ===")
# Test valid credentials and allowed action for alice
authorize_access_by_username("alice", "password", APP_MMI, PERM_WRITE)
# Test valid credentials but disallowed action for alice (e.g., Loan Management not allowed for Role A)
authorize_access_by_username("alice", "password", APP_LOANS, PERM_WRITE)
# Test invalid credentials
authorize_access_by_username("alice", "wrongpassword", APP_MMI, PERM_WRITE) # normally the password should be passed in as hash; Here we assume authentication is complete.
# Test unknown user
authorize_access_by_username("charlie", "password", APP_MMI, PERM_READ)

print("\n=== Starting Sessions ===")
# Valid session: alice using her assigned role A
session_alice = start_session("Auth001", ROLE_A)
# Invalid session: alice trying to use an unassigned role (e.g., Role C)
session_invalid = start_session("Auth001", ROLE_C)

# Valid session: alice using her another assigned role B
session_alice = start_session("Auth001", ROLE_B)

# Valid session: bob using his assigned role I
session_bob = start_session("Auth002", ROLE_I)

print("\n=== Testing Session-based Access Control ===")
# alice with a valid session: allowed permission on Money Market Instruments
authorize_session_access(session_alice, APP_MMI, PERM_WRITE)
# alice attempting an action not permitted: Loan Management Write
authorize_session_access(session_alice, APP_LOANS, PERM_WRITE)
# bob with a valid session: allowed access on Administration with Approve permission
authorize_session_access(session_bob, APP_ADMIN, PERM_APPROVE)
# bob attempting unauthorized access (e.g., Loan Management Write)
authorize_session_access(session_bob, APP_LOANS, PERM_WRITE)

print("\n=== Testing Error Handling for Session Expiry ===")
# Manually creates an expired session for testing. 
if session_alice:
    session_store[session_alice].expires_at = datetime.now() - timedelta(minutes=1)
    authorize_session_access(session_alice, APP_MMI, PERM_READ)

print("\n=== Testing Access Without Active Session ===")
# Attempt access with a non-existent session ID
authorize_session_access("non-existent-session", APP_MMI, PERM_READ)

print("\n=== Audit Log Contents ===")
for entry in access_log:
    print(entry)

# Write the audit log to a JSON file for future auditability (Task C-iii & Task E)
with open("access_log.json", "w") as f:
    json.dump(access_log, f, indent=4)

print("\nAudit log written to 'access_log.json'.")




=== Creating Users ===
Created User: Auth001, Username: alice, Position: Clerk, Function: Financial Analyst, Assigned Roles: ['B']
Created User: Auth002, Username: bob, Position: Head of Division, Function: Office Banking, Assigned Roles: []

=== Testing Access Control by Username (No Session) ===
[AUTH LOG] alice (Auth001) -> Money Market Instruments -> Write -> Access granted
[AUTH LOG] alice (Auth001) -> Loan Management -> Write -> Denied: 'Write' not allowed on 'Loan Management'
[AUTH LOG] alice (Auth001) -> Money Market Instruments -> Write -> Denied: Invalid credentials
[AUTH LOG] Denied: Unknown username 'charlie'

=== Starting Sessions ===
Session started: Auth001 with active role A. (Session ID: 44c0acba-d3d4-4d83-b569-1f6515c623e4)
ERROR: Auth001 cannot use role C. Allowed roles: {'B', 'A'}
Session started: Auth001 with active role B. (Session ID: 35aab4f5-3f0d-4825-8c30-06c20d37fd98)
Session started: Auth002 with active role I. (Session ID: 0a964da9-3233-4741-8b0e-e3a215fa67